In [49]:
!pip install efficientnet-pytorch


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.7 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=21d035b8cf4b2b4707926e32cff6d4564711c697e9b3b9411

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import cv2
import torchvision.models as models
import torch.nn as nn

# Apply CLAHE
def apply_CLAHE(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

# Apply Fourier Transform
def apply_Fourier(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.log(np.abs(fshift) + 1)
    magnitude_spectrum = (magnitude_spectrum / np.max(magnitude_spectrum) * 255).astype(np.uint8)
    return cv2.merge([magnitude_spectrum, magnitude_spectrum, magnitude_spectrum])

# Preprocessing function
def preprocess_image(image):
    image = np.array(image)

    # CLAHE and Fourier Transform
    clahe_img = apply_CLAHE(image)
    fourier_img = apply_Fourier(image)

    # Augmentation pipeline (same as during training)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    original_tensor = transform(Image.fromarray(image))
    clahe_tensor = transform(Image.fromarray(clahe_img))
    fourier_tensor = transform(Image.fromarray(fourier_img))

    # Stack all processed images (9 channels total)
    processed_tensor = torch.cat((original_tensor, clahe_tensor, fourier_tensor), dim=0)

    # Normalize 9 channels separately
    mean = torch.mean(processed_tensor, dim=[1, 2])
    std = torch.std(processed_tensor, dim=[1, 2]) + 1e-6
    processed_tensor = (processed_tensor - mean[:, None, None]) / std[:, None, None]

    return processed_tensor.unsqueeze(0)  # Add batch dimension

# Load EfficientNet-B0 and modify it for 9-channel input
class ModifiedEfficientNet(nn.Module):
    def __init__(self, original_model, num_classes):
        super(ModifiedEfficientNet, self).__init__()
        self.channel_reduction = nn.Conv2d(9, 3, kernel_size=1)  # Reduce 9 → 3 channels
        self.efficient_net = original_model

        # Modify classifier (same as training)
        self.efficient_net.classifier = nn.Sequential(
            nn.Linear(self.efficient_net.classifier[1].in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),  # Same dropout value as in training
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.channel_reduction(x)
        x = self.efficient_net(x)
        return x

# Load the trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/content/drive/MyDrive/PROJECT/MODEL.pth"

# Load EfficientNet-B0 with ImageNet weights
base_model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# Create modified model
num_classes = 7
model = ModifiedEfficientNet(base_model, num_classes=num_classes)

# Load the trained state_dict
state_dict = torch.load(model_path, map_location=device)

# Fix mismatched keys (remove 'module.' if trained with DataParallel)
new_state_dict = {}
for k, v in state_dict.items():
    new_key = k.replace("module.", "")
    new_state_dict[new_key] = v

model.load_state_dict(new_state_dict)
model = model.to(device)
model.eval()

# Define class names (in correct order)
class_names = [
    'dyed_lifted_polyps',
    'dyed_resection_margins',
    'esophagus',
    'normal_pylorus',
    'normal_z_line',
    'polyps',
    'ulcerative_colitis'
]

# Prediction function
def predict(image_path):
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess_image(image).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.softmax(output, dim=1)[0]
        predicted_class = torch.argmax(probabilities).item()

    print(f"✅ Prediction: {class_names[predicted_class]}")
    print(f"🔥 Confidence: {probabilities[predicted_class].item() * 100:.2f}%")
    return class_names[predicted_class], probabilities.cpu().numpy()

# Example Inference
image_path = "/content/drive/MyDrive/PROJECT/images (2).jpg"  #Replace with actual path
predicted_class, probabilities = predict(image_path)


✅ Prediction: ulcerative_colitis
🔥 Confidence: 99.98%
